<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
#how many launches
spacex_df.shape

(56, 4)

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
#initial coordinates
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Iterate over the DataFrame rows to add circles and markers
for index, row in launch_sites_df.iterrows():
    # Add a circle for each launch site
    circle = folium.Circle(
        location=(row['Lat'], row['Long']),
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))

    # Add a marker for each launch site
    marker = folium.Marker(
        location=(row['Lat'], row['Long']),
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["Launch Site"]}</b></div>',
        )
    )

    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map



The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [10]:
spacex_df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [11]:
# Task 2: Mark the success/failed launches for each site on the map
site_map = folium.Map(location = nasa_coordinate, zoom_start = 5)
# Iterate over the DataFrame rows to add circles and markers
for index, row in spacex_df.iterrows():
    # Add a circle for each launch site
    circle = folium.Circle(
        location=(row['Lat'], row['Long']),
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))

    # Add a marker for each launch site
    marker = folium.Marker(
        location=(row['Lat'], row['Long']),
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["class"]}</b></div>',
        )
    )

    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# assigh color column to spacex_df
color_list = []
for i, row in spacex_df.iterrows():
    if row['class'] == 1:
        color = 'green'
    else:
        color = 'red'
    color_list.append(color)
color_list[46:56]
# add column with color code to spacex_df
spacex_df['marker_color'] = color_list
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(location=(record['Lat'], record['Long']), icon=folium.Icon( color ='white',
            icon_size=(20, 20),
            icon_anchor=(0, 0), icon_color = record['marker_color'],
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["class"]}</b></div>',
        )
    )
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [14]:
# TASK 3: Calculate the distances between a launch site to its proximities
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [17]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
coastline = [28.56538, -80.56721]
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance = []
for i, row in launch_sites_df.iterrows():
    distance_coastline = calculate_distance(row['Lat'], row['Long'], coastline[0], coastline[1])
    distance.append(distance_coastline)
distance[0:5]
    

[1.0486861931021147, 0.9697015840164972, 7.833292010366386, 3827.849809965002]

In [20]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [18]:
launch_sites_df.loc[1,'Launch Site']

'CCAFS SLC-40'

In [19]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance1 = distance[1]
coordinate = (launch_sites_df.loc[1,'Lat'], launch_sites_df.loc[1,'Long'])
site_name = launch_sites_df.loc[1,'Launch Site']
site_map_CCAFSSLC = folium.Map(location = coordinate, zoom_start = 10)
circle = folium.Circle(location = coordinate, color='#000000',
        fill=True
    ).add_child(folium.Popup(site_name))
# Create a Marker object with both the distance and site name displayed
distance_marker = folium.Marker(
    location=coordinate,
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'''
        <div style="font-size: 12px; color:#d35400;">
            <b>{site_name}</b><br>
            <b>{distance1:.2f} KM</b>
        </div>
        ''',
    )
)


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [54]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
#lines=folium.PolyLine(locations=[coordinate,coastline], weight=1)
#site_map_CCAFSSLC.add_child(lines)

In [20]:
# Add the marker to the map
site_map_CCAFSSLC.add_child(distance_marker)
site_map_CCAFSSLC.add_child(circle)

# Create a PolyLine object to connect the launch site and coastline
lines = folium.PolyLine(locations=[coordinate, coastline], weight=1)

# Add the PolyLine to the map
site_map_CCAFSSLC.add_child(lines)

# Display the map
site_map_CCAFSSLC


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [24]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [43]:
coastlines_df = pd.DataFrame({
    'Landmark': ['East Coast', 'West Coast'],
    'Lat': [28.56538, 34.63688],
    'Long': [-80.56721, -120.6249]
})

railways_df = pd.DataFrame({
    'Landmark': ['Railway East', 'Railway West'],
    'Lat': [28.57113, 34.6366],
    'Long': [-80.84541, -120.62327]
})

city_df = pd.DataFrame({
    'Landmark': ['Los angeles', 'Orlando'],
    'Lat': [28.59417, 34.04811],
    'Long': [-81.38672, -118.2225]
})

highway_df = pd.DataFrame({
    'Landmark': ['Highway East', 'Highway West'],
    'Lat': [28.55765, 34.67921],
    'Long': [-80.83311, -120.45672]
})

In [34]:
coastlines_df
coastlines_df.loc[0, 'Lat']

28.56538

In [44]:
#calculate distance to coast
distance_east_coast = []
distance_west_coast = []
for i, row in launch_sites_df.iterrows():
    dis_east = calculate_distance(row['Lat'], row['Long'], coastlines_df.loc[0, 'Lat'], coastlines_df.loc[0, 'Long'])
    dis_west = calculate_distance(row['Lat'], row['Long'], coastlines_df.loc[1, 'Lat'], coastlines_df.loc[1, 'Long'])
    distance_east_coast.append(dis_east)
    distance_west_coast.append(dis_west)
print('East coast:', distance_east_coast)
print('West coast:', distance_west_coast)
                          
#calculate distance to railway
distance_east_rail = []
distance_west_rail = []
for i, row in launch_sites_df.iterrows():
    dis_east = calculate_distance(row['Lat'], row['Long'], railways_df.loc[0, 'Lat'], railways_df.loc[0, 'Long'])
    dis_west = calculate_distance(row['Lat'], row['Long'], railways_df.loc[1, 'Lat'], railways_df.loc[1, 'Long'])
    distance_east_rail.append(dis_east)
    distance_west_rail.append(dis_west)
print('East railway:', distance_east_rail)
print('West railway:', distance_west_rail)

#calculate distance to highway
distance_east_highway = []
distance_west_highway = []
for i, row in launch_sites_df.iterrows():
    dis_east = calculate_distance(row['Lat'], row['Long'], highway_df.loc[0, 'Lat'], highway_df.loc[0, 'Long'])
    dis_west = calculate_distance(row['Lat'], row['Long'], highway_df.loc[1, 'Lat'], highway_df.loc[1, 'Long'])
    distance_east_highway.append(dis_east)
    distance_west_highway.append(dis_west)
print('East highway:', distance_east_highway)
print('West highway:', distance_west_highway)

#calculate distance to city
distance_east_city = []
distance_west_city = []
for i, row in launch_sites_df.iterrows():
    dis_east = calculate_distance(row['Lat'], row['Long'], city_df.loc[0, 'Lat'], city_df.loc[0, 'Long'])
    dis_west = calculate_distance(row['Lat'], row['Long'], city_df.loc[1, 'Lat'], city_df.loc[1, 'Long'])
    distance_east_city.append(dis_east)
    distance_west_city.append(dis_west)
print('East city:', distance_east_city)
print('West city:', distance_west_city)

East coast: [1.0486861931021147, 0.9697015840164972, 7.833292010366386, 3827.849809965002]
West coast: [3828.327507071404, 3828.3416999867172, 3821.5380552535216, 1.3713452276194031]
East railway: [26.20422938758474, 26.252950970280946, 19.393094996739624, 3802.1692237066236]
West railway: [3828.178983031538, 3828.193177003703, 3821.3895194893353, 1.2203682769603756]
East highway: [24.991210661836334, 25.04575669772976, 18.273927278928053, 3803.8197402049377]
West highway: [3812.8555443158393, 3812.8695999452193, 3806.067622818015, 15.007044386129348]
East city: [79.13637956948351, 79.18399521859115, 72.29824656265052, 3751.7378088102982]
West city: [3608.7786401745493, 3608.795025626672, 3601.9661413306385, 228.77478632665753]


In [27]:
#find index of min value of landmark
min_dis = min(distance_east_coast)
index_min = 0
site = 'a'
dist = 0
dic = {}
dic['site'] = []
for i, dis in enumerate(distance_east_coast):
    if dis == min_dis:
        dist = dis
        index_min= i
        site = launch_sites_df.loc[index_min, 'Launch Site']
        dic['site'].append(dist)
print(index_min, site, dist)
dic
    


1 CCAFS SLC-40 0.9697015840164972


{'site': [0.9697015840164972]}

#dataframe of landmarks coordinates
coastlines_df = pd.DataFrame({
    'Landmark': ['East Coast', 'West Coast'],
    'Lat': [28.56538, 34.63688],
    'Long': [-80.56721, -120.6249]
})

railways_df = pd.DataFrame({
    'Landmark': ['Railway East', 'Railway West'],
    'Lat': [28.57113, 34.6366],
    'Long': [-80.84541, -120.62327]
})

In [45]:
# Dictionary to map landmarks to their distance lists
landmarks = {
    'East Coast': distance_east_coast,
    'West Coast': distance_west_coast,
    'Railway East': distance_east_rail,
    'Railway West': distance_west_rail,
    'Highway East': distance_east_highway,
    'Highway West': distance_west_highway,
    'Los angeles': distance_east_city,
    'Orlando': distance_west_city
    # Add more landmarks as needed
}

# Initialize dictionary to store results
min_distances = {'Launch Site': [], 'Closest Landmark': [], 'Distance': []}

# Loop through each landmark and calculate the minimum distance
for landmark_name, distance_list in landmarks.items():
    min_dis = min(distance_list)
    index_min = distance_list.index(min_dis)
    site = launch_sites_df.loc[index_min, 'Launch Site']

    # Store the results
    min_distances['Launch Site'].append(site)
    min_distances['Closest Landmark'].append(landmark_name)
    min_distances['Distance'].append(min_dis)

# Convert the dictionary to a DataFrame
min_distances_df = pd.DataFrame(min_distances)

print(min_distances_df)

    Launch Site Closest Landmark    Distance
0  CCAFS SLC-40       East Coast    0.969702
1   VAFB SLC-4E       West Coast    1.371345
2    KSC LC-39A     Railway East   19.393095
3   VAFB SLC-4E     Railway West    1.220368
4    KSC LC-39A     Highway East   18.273927
5   VAFB SLC-4E     Highway West   15.007044
6    KSC LC-39A      Los angeles   72.298247
7   VAFB SLC-4E          Orlando  228.774786


In [38]:
coastlines_df

,Landmark,Lat,Long
0,East Coast,28.56538,-80.56721
1,West Coast,34.63688,-120.62490


#add landmark coordinates wth loop option
coor = {'Lat':[], 'Long':[]}
for i, row in min_distances_df.iterrows():
    for j, item in coastlines_df.iterrows():
        if item['Landmark'] == row['Closest Landmark']:
            lat = item['Lat']
            long = item['Long']
            coor['Lat'].append(lat)
            coor['Long'].append(long)
coor
            
            

In [46]:
#easier approach is to combine the landmarks into one dataframe and merge landmark to min_distances dataframe.
# Combine coastlines and railways into a single DataFrame
landmarks_df = pd.concat([coastlines_df, railways_df, highway_df, city_df], ignore_index=True)

# Merge the min_distances_df with landmarks_df on the 'Closest Landmark' and 'Landmark' columns
min_distances_with_coords = min_distances_df.merge(landmarks_df, how='left', left_on='Closest Landmark', right_on='Landmark')

# Drop the redundant 'Landmark' column after the merge
min_distances_with_coords = min_distances_with_coords.drop(columns=['Landmark'])

print(min_distances_with_coords)


    Launch Site Closest Landmark    Distance       Lat       Long
0  CCAFS SLC-40       East Coast    0.969702  28.56538  -80.56721
1   VAFB SLC-4E       West Coast    1.371345  34.63688 -120.62490
2    KSC LC-39A     Railway East   19.393095  28.57113  -80.84541
3   VAFB SLC-4E     Railway West    1.220368  34.63660 -120.62327
4    KSC LC-39A     Highway East   18.273927  28.55765  -80.83311
5   VAFB SLC-4E     Highway West   15.007044  34.67921 -120.45672
6    KSC LC-39A      Los angeles   72.298247  28.59417  -81.38672
7   VAFB SLC-4E          Orlando  228.774786  34.04811 -118.22250


In [48]:
# Example DataFrames
#launch_sites_df = pd.DataFrame({
#   'Launch Site': ['Site 1', 'Site 2'],
#    'Lat': [28.5623, 34.6328],
#    'Long': [-80.5774, -120.6108]
#})

#coastlines_df = pd.DataFrame({
#    'Coastline Name': ['East coast', 'West coast'],
#    'Lat': [28.56538, 34.63688],
#    'Long': [-80.56721, -120.6249]
#})

#railways_df = pd.DataFrame({
#    'Railway Name': ['Railway East', 'Railway West'],
#    'Lat': [28.57113, 34.6366],
#    'Long': [-80.84541, -120.62327]
#})

# Create the folium map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Add Launch Sites
for _, site in launch_sites_df.iterrows():
    folium.Marker(
        location=(site['Lat'], site['Long']),
        popup=f"Launch Site: {site['Launch Site']}",
        icon=folium.Icon(color='blue', icon='rocket')
    ).add_to(site_map)

# Add Coastlines
for _, coastline in coastlines_df.iterrows():
    folium.Marker(
        location=(coastline['Lat'], coastline['Long']),
        popup=f"Coastline: {coastline['Landmark']}",
        icon=folium.Icon(color='green', icon='tint')
    ).add_to(site_map)

# Add Railways
for _, railway in railways_df.iterrows():
    folium.Marker(
        location=(railway['Lat'], railway['Long']),
        popup=f"Railway: {railway['Landmark']}",
        icon=folium.Icon(color='red', icon='train')
    ).add_to(site_map)

# Add Highway
for _, highway in highway_df.iterrows():
    folium.Marker(
        location=(highway['Lat'], highway['Long']),
        popup=f"Highway: {highway['Landmark']}",
        icon=folium.Icon(color='orange', icon='road')
    ).add_to(site_map)

# Add City
for _, city in city_df.iterrows():
    folium.Marker(
        location=(city['Lat'], city['Long']),
        popup=f"City: {city['Landmark']}",
        icon=folium.Icon(color='purple', icon='city')
    ).add_to(site_map)



# Display the map
site_map

In [49]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [50]:
min_distances_with_coords

,Launch Site,Closest Landmark,Distance,Lat,Long
0,CCAFS SLC-40,East Coast,0.969702,28.56538,-80.56721
1,VAFB SLC-4E,West Coast,1.371345,34.63688,-120.62490
2,KSC LC-39A,Railway East,19.393095,28.57113,-80.84541
3,VAFB SLC-4E,Railway West,1.220368,34.63660,-120.62327
4,KSC LC-39A,Highway East,18.273927,28.55765,-80.83311
5,VAFB SLC-4E,Highway West,15.007044,34.67921,-120.45672
6,KSC LC-39A,Los angeles,72.298247,28.59417,-81.38672
7,VAFB SLC-4E,Orlando,228.774786,34.04811,-118.22250


In [53]:
# Add lines from each launch site to its closest landmark
# Add lines and labels from each launch site to its closest landmark
for _, row in min_distances_with_coords.iterrows():
    # Get the launch site location
    launch_site_location = launch_sites_df[launch_sites_df['Launch Site'] == row['Launch Site']].iloc[0]

    # Create a PolyLine connecting the launch site to the closest landmark
    folium.PolyLine(
        locations=[(launch_site_location['Lat'], launch_site_location['Long']), (row['Lat'], row['Long'])],
        color='white'
    ).add_to(site_map)

    # Calculate the midpoint for the label
    midpoint = (
        (launch_site_location['Lat'] + row['Lat']) / 2,
        (launch_site_location['Long'] + row['Long']) / 2
    )

    # Create a distance label at the midpoint
    folium.Marker(
        location=midpoint,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: blue;">{row["Distance"]:.2f} km</div>',
        )
    ).add_to(site_map)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


#Are launch sites in close proximity to railways?
yes railway can be found within 20 km distance from the launch sites.

#Are launch sites in close proximity to highways?
yes highways can be found approx. 15 - 20 km away from launch sites.

#Are launch sites in close proximity to coastline?
yes coastlines can be found < 2km away from launch sites.

#Do launch sites keep certain distance away from cities?
yes, launchsites are > 50 km away from cities.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
